In [40]:
import tsfresh
import os
import pandas as pd
from tsfresh import extract_relevant_features
from multiprocessing import freeze_support
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction.settings import MinimalFCParameters
from sklearn.model_selection import train_test_split

In [41]:
import numpy as np
freeze_support()

#names = ["v"+str(x) for x in range(1,10)]+["d"]+["c"]
data = pd.read_csv("train_data.csv" ,sep=",",index_col=0)
data = data.fillna(value = 100)
data_size = len(data)
data

,u_0,u_1,u_2,u_3,u_4,u_5,u_6,u_7,u_8,u_9,...,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,cat
83,0.091048,0.13656,0.15830,0.15397,0.12417,0.072242,0.004004,-0.072858,-0.149550,-0.21703,...,-0.234460,-0.238830,-0.228490,-0.206180,-0.175500,-0.140540,-0.105520,-0.074317,-0.050134,hammerstein
52,1.027500,0.90393,0.80091,0.72354,0.67431,0.652840,0.655940,0.677970,0.711390,0.74754,...,0.986980,0.969900,0.946350,0.916380,0.880780,0.841000,0.799000,0.757040,0.717300,hammerstein
364,-1.830800,-1.86180,-1.85860,-1.82200,-1.75520,-1.664200,-1.556600,-1.441500,-1.328000,-1.22470,...,-0.428950,-0.433940,-0.435500,-0.433980,-0.429650,-0.422760,-0.413570,-0.402440,-0.389910,wiener
491,0.499200,0.44394,0.40211,0.38056,0.38397,0.414370,0.470960,0.550140,0.645840,0.75013,...,0.122740,0.169010,0.210580,0.245590,0.272900,0.292220,0.304120,0.310000,0.311880,nde
254,0.787180,0.75546,0.69580,0.60906,0.49887,0.371330,0.234420,0.097206,-0.030973,-0.14145,...,0.296470,0.303420,0.305480,0.301380,0.290560,0.273070,0.249250,0.219510,0.184210,wiener
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,-0.217030,-0.26692,-0.29233,-0.28855,-0.25353,-0.188080,-0.095828,0.017123,0.142780,0.27201,...,0.060658,0.052231,0.033291,0.006109,-0.026121,-0.059422,-0.089257,-0.110990,-0.120610,wiener
54,0.800910,0.72354,0.67431,0.65284,0.65594,0.677970,0.711390,0.747540,0.777490,0.79298,...,0.946350,0.916380,0.880780,0.841000,0.799000,0.757040,0.717300,0.681650,0.651400,hammerstein
431,1.164300,1.02750,0.90393,0.80091,0.72354,0.674310,0.652840,0.655940,0.677970,0.71139,...,1.320100,1.342700,1.354500,1.353800,1.339500,1.311800,1.271800,1.221600,1.164000,nde
151,-0.455680,-0.61401,-0.75714,-0.87776,-0.97072,-1.033500,-1.066600,-1.073000,-1.058500,-1.03050,...,0.116320,0.056185,-0.016790,-0.100690,-0.192710,-0.289520,-0.387590,-0.483480,-0.574060,hammerstein


In [42]:
mydata = data.transpose()
my_y = data.iloc[:,-2]
mydata
my_y

83    -0.050134
52     0.717300
364   -0.389910
491    0.311880
254    0.184210
         ...   
282   -0.120610
54     0.651400
431    1.164000
151   -0.574060
243    0.267660
Name: y_9, Length: 285, dtype: float64

In [43]:
mydata = mydata.drop(mydata.index[[-1,-2]])
mydata
len(mydata)

19

In [44]:
new = mydata.values.transpose().reshape(-1,1)
new.shape

(5415, 1)

In [45]:
id_list = []
for x in range(data_size):
    id_list.extend(len(mydata)*[x])
my_id = pd.Series(id_list)
my_id

0         0
1         0
2         0
3         0
4         0
       ... 
5410    284
5411    284
5412    284
5413    284
5414    284
Length: 5415, dtype: int64

In [46]:
tsdata = pd.DataFrame()
tsdata["id"] = my_id
tsdata["time"] = data_size*[x for x in range(len(mydata))]
tsdata["value"] = new
tsdata

,id,time,value
0,0,0,0.091048
1,0,1,0.13656
2,0,2,0.1583
3,0,3,0.15397
4,0,4,0.12417
...,...,...,...
5410,284,14,0.28896
5411,284,15,0.27593
5412,284,16,0.26676
5413,284,17,0.26222


In [47]:
extracted_features = extract_features(tsdata, column_id="id", column_sort="time", default_fc_parameters=MinimalFCParameters())

Feature Extraction: 100%|██████████| 36/36 [00:01<00:00, 19.02it/s]


In [48]:
features = extracted_features.dropna(axis=1, how="any")

In [49]:
selected = select_features(features, my_y.values, ml_task = "regression", fdr_level=0.1 )

selected

,value__sum_values,value__mean,value__median,value__maximum,value__minimum,value__root_mean_square
0,-1.316731,-0.069302,-0.105520,0.15830,-0.238830,0.160716
1,15.671420,0.824812,0.800910,1.02750,0.652840,0.833966
2,-20.164360,-1.061282,-1.224700,-0.40244,-1.861800,1.231816
3,6.942598,0.365400,0.380560,0.75013,0.074218,0.402522
4,6.402663,0.336982,0.296470,0.78718,-0.141450,0.412512
...,...,...,...,...,...,...
280,-1.246450,-0.065603,-0.059422,0.27201,-0.292330,0.166373
281,14.724310,0.774964,0.757040,0.96990,0.652840,0.780814
282,19.797630,1.041981,1.164300,1.35450,0.652840,1.080208
283,-10.072715,-0.530143,-0.483480,0.16297,-1.073000,0.687050


In [50]:
finaldata = selected
finaldata["y"] = my_y.values
finaldata



,value__sum_values,value__mean,value__median,value__maximum,value__minimum,value__root_mean_square,y
0,-1.316731,-0.069302,-0.105520,0.15830,-0.238830,0.160716,-0.050134
1,15.671420,0.824812,0.800910,1.02750,0.652840,0.833966,0.717300
2,-20.164360,-1.061282,-1.224700,-0.40244,-1.861800,1.231816,-0.389910
3,6.942598,0.365400,0.380560,0.75013,0.074218,0.402522,0.311880
4,6.402663,0.336982,0.296470,0.78718,-0.141450,0.412512,0.184210
...,...,...,...,...,...,...,...
280,-1.246450,-0.065603,-0.059422,0.27201,-0.292330,0.166373,-0.120610
281,14.724310,0.774964,0.757040,0.96990,0.652840,0.780814,0.651400
282,19.797630,1.041981,1.164300,1.35450,0.652840,1.080208,1.164000
283,-10.072715,-0.530143,-0.483480,0.16297,-1.073000,0.687050,-0.574060


In [52]:
train, test = train_test_split(finaldata, test_size=0.5)
#print(test.head())
train.to_csv(path_or_buf="train_data_ts.csv", header=list(finaldata.columns), index=True)
test.to_csv(path_or_buf="test_data_ts.csv", header=list(finaldata.columns), index=True)

In [55]:
with open("data_ts_desc.txt", "w") as file:
    file.write("train_data_ts.csv")
    file.write("\nNA")
    file.write("\n2")
    file.write("\n1 index x")
    for count,i in enumerate(list(finaldata.columns)[:-1]):
        file.write("\n"+str(count+2)+" "+str(i)+" n")
    file.write("\n"+str(finaldata.shape[1]+1)+ " y d")